In this project, I will use machine learning models to predict which baseball players will be voted into the Hall of Fame based on players' career statistics in Lahman's Baseball database.

Agenda:
* importing data
* data cleaning and preprocessing
* feature and target columns
* logistic regression
* error metric
* conclusion

**Importing data**

In [1]:
import pandas as pd

In [4]:
master_df = pd.read_csv('core/Master.csv',usecols=['playerID','nameFirst','nameLast','bats','throws','debut','finalGame'])
fielding_df = pd.read_csv('core/Fielding.csv',usecols=['playerID','yearID','stint','teamID','lgID','POS','G','GS','InnOuts','PO','A','E','DP'])
batting_df = pd.read_csv('core/Batting.csv')
awards_df = pd.read_csv('core/AwardsPlayers.csv', usecols=['playerID','awardID','yearID'])
allstar_df = pd.read_csv('core/AllstarFull.csv', usecols=['playerID','yearID'])
hof_df = pd.read_csv('core/HallOfFame.csv',usecols=['playerID','yearid','votedBy','needed_note','inducted','category'])
appearances_df = pd.read_csv('core/Appearances.csv')

**Data cleaning and preprocessing**

batting_df

In [5]:
batting_df.head()

,playerID,yearID,stint,teamID,lgID,G,AB,R,H,2B,...,RBI,SB,CS,BB,SO,IBB,HBP,SH,SF,GIDP
0,abercda01,1871,1,TRO,NaN,1,4,0,0,0,...,0.0,0.0,0.0,0,0.0,NaN,NaN,NaN,NaN,NaN
1,addybo01,1871,1,RC1,NaN,25,118,30,32,6,...,13.0,8.0,1.0,4,0.0,NaN,NaN,NaN,NaN,NaN
2,allisar01,1871,1,CL1,NaN,29,137,28,40,4,...,19.0,3.0,1.0,2,5.0,NaN,NaN,NaN,NaN,NaN
3,allisdo01,1871,1,WS3,NaN,27,133,28,44,10,...,27.0,1.0,1.0,0,2.0,NaN,NaN,NaN,NaN,NaN
4,ansonca01,1871,1,RC1,NaN,25,120,29,39,11,...,16.0,6.0,2.0,2,1.0,NaN,NaN,NaN,NaN,NaN


The data is not tidy. I want to make sure that each row is an observation, in this case a player's career, and each column is a variable.

Thus, I will create dictionaries for players and aggreagate their statistics.

In [7]:
player_stats = {} #key: playerID  value: player stats
years_played = {} #key: playerID  value: yearID

for i, row in batting_df.iterrows():
    playerID = row['playerID']
    if playerID in player_stats:
        player_stats[playerID]['G'] = player_stats[playerID]['G'] + row['G']
        player_stats[playerID]['AB'] = player_stats[playerID]['AB'] + row['AB']
        player_stats[playerID]['R'] = player_stats[playerID]['R'] + row['R']
        player_stats[playerID]['H'] = player_stats[playerID]['H'] + row['H']
        player_stats[playerID]['2B'] = player_stats[playerID]['2B'] + row['2B']
        player_stats[playerID]['3B'] = player_stats[playerID]['3B'] + row['3B']
        player_stats[playerID]['HR'] = player_stats[playerID]['HR'] + row['HR']
        player_stats[playerID]['RBI'] = player_stats[playerID]['RBI'] + row['RBI']
        player_stats[playerID]['SB'] = player_stats[playerID]['SB'] + row['SB']
        player_stats[playerID]['BB'] = player_stats[playerID]['BB'] + row['BB']
        player_stats[playerID]['SO'] = player_stats[playerID]['SO'] + row['SO']
        player_stats[playerID]['IBB'] = player_stats[playerID]['IBB'] + row['IBB']
        player_stats[playerID]['HBP'] = player_stats[playerID]['HBP'] + row['HBP']
        player_stats[playerID]['SH'] = player_stats[playerID]['SH'] + row['SH']
        player_stats[playerID]['SF'] = player_stats[playerID]['SF'] + row['SF']
        years_played[playerID].append(row['yearID'])
    else:
        player_stats[playerID] = {}
        player_stats[playerID]['G'] = row['G']
        player_stats[playerID]['AB'] = row['AB']
        player_stats[playerID]['R'] = row['R']
        player_stats[playerID]['H'] = row['H']
        player_stats[playerID]['2B'] = row['2B']
        player_stats[playerID]['3B'] = row['3B']
        player_stats[playerID]['HR'] = row['HR']
        player_stats[playerID]['RBI'] = row['RBI']
        player_stats[playerID]['SB'] = row['SB']
        player_stats[playerID]['BB'] = row['BB']
        player_stats[playerID]['SO'] = row['SO']
        player_stats[playerID]['IBB'] = row['IBB']
        player_stats[playerID]['HBP'] = row['HBP']
        player_stats[playerID]['SH'] = row['SH']
        player_stats[playerID]['SF'] = row['SF']
        years_played[playerID] = []
        years_played[playerID].append(row['yearID'])

add number of years_played data into the player_stats dictionary

In [8]:
for k,v in years_played.items():
    player_stats[k]['Years_Played'] = len(list(set(v)))

fielding_df

aggregate players' data from fielding_df into player_stats dictionary

In [13]:
fielder_list = []

for i,row in fielding_df.iterrows():
    playerID = row['playerID']
    Gf = row['G']
    GSf = row['GS']
    POf = row['PO']
    Af = row['A']
    Ef = row['E']
    DPf = row['DP']
    
    if playerID in player_stats and playerID in fielder_list:
        player_stats[playerID]['Gf'] = player_stats[playerID]['Gf'] + Gf
        player_stats[playerID]['GSf'] = player_stats[playerID]['GSf'] + GSf
        player_stats[playerID]['POf'] = player_stats[playerID]['POf'] + POf
        player_stats[playerID]['Af'] = player_stats[playerID]['Af'] + Af
        player_stats[playerID]['Ef'] = player_stats[playerID]['Ef'] + Ef
        player_stats[playerID]['DPf'] = player_stats[playerID]['DPf'] + DPf
    else:
        fielder_list.append(playerID)
        player_stats[playerID]['Gf'] = Gf
        player_stats[playerID]['GSf'] = GSf
        player_stats[playerID]['POf'] = POf
        player_stats[playerID]['Af'] = Af
        player_stats[playerID]['Ef'] = Ef
        player_stats[playerID]['DPf'] = DPf

awards_df

In [14]:
awards_df['awardID'].unique()

array(['Pitching Triple Crown', 'Triple Crown',
       'Baseball Magazine All-Star', 'Most Valuable Player',
       'TSN All-Star', 'TSN Guide MVP',
       'TSN Major League Player of the Year', 'TSN Pitcher of the Year',
       'TSN Player of the Year', 'Rookie of the Year', 'Babe Ruth Award',
       'Lou Gehrig Memorial Award', 'World Series MVP', 'Cy Young Award',
       'Gold Glove', 'TSN Fireman of the Year', 'All-Star Game MVP',
       'Hutch Award', 'Roberto Clemente Award',
       'Rolaids Relief Man Award', 'NLCS MVP', 'ALCS MVP',
       'Silver Slugger', 'Branch Rickey Award', 'Hank Aaron Award',
       'TSN Reliever of the Year', 'Comeback Player of the Year',
       'Outstanding DH Award', 'Reliever of the Year Award'], dtype=object)

select 5 of the awardID to be included in the model

In [15]:
mvp = awards_df[awards_df['awardID'] == 'Most Valuable Player']
roy = awards_df[awards_df['awardID'] == 'Rookie of the Year']
gg = awards_df[awards_df['awardID'] == 'Gold Glove']
ss = awards_df[awards_df['awardID'] == 'Silver Slugger']
ws_mvp = awards_df[awards_df['awardID'] == 'World Series MVP']
awards_list = [mvp,roy,gg,ss,ws_mvp]

In [18]:
mvp_list = []
roy_list = []
gg_list = []
ss_list = []
ws_mvp_list = []
lists = [mvp_list,roy_list,gg_list,ss_list,ws_mvp_list]

add awards count for each player in player_stats

In [20]:
for index,v in enumerate(awards_list):
    for i,row in v.iterrows():
        playerID = row['playerID']
        award = row['awardID']
        
        if playerID in player_stats and playerID in lists[index]:
            player_stats[playerID][award] += 1
        else:
            lists[index].append(playerID)
            player_stats[playerID][award] = 1

allstar_df

add a count of appearences in Allstar game for players in player_stats

In [21]:
allstar_list = []

for i,row in allstar_df.iterrows():
    playerID = row['playerID']
    if playerID in player_stats and playerID in allstar_list:
        player_stats[playerID]['AS_games'] += 1
    else:
        allstar_list.append(playerID)
        player_stats[playerID]['AS_games'] = 1

hof_df

In [22]:
hof_df = hof_df[(hof_df['inducted'] == 'Y') & (hof_df['category'] == 'Player')]

In [23]:
for i,row in hof_df.iterrows():
    playerID = row['playerID']
    if playerID in player_stats:
        player_stats[playerID]['HoF'] = 1
        player_stats[playerID]['votedBy'] = row['votedBy']

player_stats

Until now, I have collected data from batting_df, fielding_df, awards_df, 
allstar_df, and hof_df into the player_stats dictionary. Then, I will 
convert it to a dataframe stats_df.

In [31]:
stats_df = pd.DataFrame.from_dict(player_stats,orient='index')
stats_df.head()

,G,AB,R,H,2B,3B,HR,RBI,SB,BB,...,Ef,DPf,HoF,votedBy,Most Valuable Player,AS_games,Gold Glove,Rookie of the Year,World Series MVP,Silver Slugger
aardsda01,331,4,0,0,0,0,0,0.0,0.0,0,...,3.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
aaronha01,3298,12364,2174,3771,624,98,755,2297.0,240.0,1402,...,144.0,218.0,1.0,BBWAA,1.0,25.0,3.0,NaN,NaN,NaN
aaronto01,437,944,102,216,42,6,13,94.0,9.0,86,...,22.0,124.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
aasedo01,448,5,0,0,0,0,0,0.0,0.0,0,...,13.0,10.0,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN
abadan01,15,21,1,2,0,0,0,0.0,0.0,4,...,1.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


join stats_df with master_df on index playerID

In [33]:
stats_df['playerID'] = stats_df.index
master_df = pd.read_csv('core/Master.csv',usecols=['playerID','nameFirst','nameLast','bats','throws','debut','finalGame'])
master_df = master_df.join(stats_df, on='playerID', how='inner',
                        rsuffix='mstr')
master_df.head()

,playerID,nameFirst,nameLast,bats,throws,debut,finalGame,G,AB,R,...,DPf,HoF,votedBy,Most Valuable Player,AS_games,Gold Glove,Rookie of the Year,World Series MVP,Silver Slugger,playerIDmstr
0,aardsda01,David,Aardsma,R,R,2004-04-06,2015-08-23,331,4,0,...,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,aardsda01
1,aaronha01,Hank,Aaron,R,R,1954-04-13,1976-10-03,3298,12364,2174,...,218.0,1.0,BBWAA,1.0,25.0,3.0,NaN,NaN,NaN,aaronha01
2,aaronto01,Tommie,Aaron,R,R,1962-04-10,1971-09-26,437,944,102,...,124.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,aaronto01
3,aasedo01,Don,Aase,R,R,1977-07-26,1990-10-03,448,5,0,...,10.0,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,aasedo01
4,abadan01,Andy,Abad,L,L,2001-09-10,2006-04-13,15,21,1,...,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,abadan01


appearances_df

In [34]:
appearances_df.head()

,yearID,teamID,lgID,playerID,G_all,GS,G_batting,G_defense,G_p,G_c,...,G_2b,G_3b,G_ss,G_lf,G_cf,G_rf,G_of,G_dh,G_ph,G_pr
0,1871,TRO,NaN,abercda01,1,NaN,1,1,0,0,...,0,0,1,0,0,0,0,NaN,NaN,NaN
1,1871,RC1,NaN,addybo01,25,NaN,25,25,0,0,...,22,0,3,0,0,0,0,NaN,NaN,NaN
2,1871,CL1,NaN,allisar01,29,NaN,29,29,0,0,...,2,0,0,0,29,0,29,NaN,NaN,NaN
3,1871,WS3,NaN,allisdo01,27,NaN,27,27,0,27,...,0,0,0,0,0,0,0,NaN,NaN,NaN
4,1871,RC1,NaN,ansonca01,25,NaN,25,25,0,5,...,2,20,0,1,0,0,1,NaN,NaN,NaN


http://www.seanlahman.com/files/database/readme2016.txt

aggregate information about appearances at each position for players

aggregate number of games played in each era for players

In [36]:
pos_dict = {}

for i,row in appearances_df.iterrows():
    ID = row['playerID']
    year = row['yearID']
    if ID in pos_dict:
        pos_dict[ID]['G_all'] = pos_dict[ID]['G_all'] + row['G_all']
        pos_dict[ID]['G_p'] = pos_dict[ID]['G_p'] + row['G_p']
        pos_dict[ID]['G_c'] = pos_dict[ID]['G_c'] + row['G_c']
        pos_dict[ID]['G_1b'] = pos_dict[ID]['G_1b'] + row['G_1b']
        pos_dict[ID]['G_2b'] = pos_dict[ID]['G_2b'] + row['G_2b']
        pos_dict[ID]['G_3b'] = pos_dict[ID]['G_3b'] + row['G_3b']
        pos_dict[ID]['G_ss'] = pos_dict[ID]['G_ss'] + row['G_ss']
        pos_dict[ID]['G_lf'] = pos_dict[ID]['G_lf'] + row['G_lf']
        pos_dict[ID]['G_cf'] = pos_dict[ID]['G_cf'] + row['G_cf']
        pos_dict[ID]['G_rf'] = pos_dict[ID]['G_rf'] + row['G_rf']
        pos_dict[ID]['G_of'] = pos_dict[ID]['G_of'] + row['G_of']
        pos_dict[ID]['G_dh'] = pos_dict[ID]['G_dh'] + row['G_dh']
        
        if year < 1920:
            pos_dict[ID]['pre1920'] = pos_dict[ID]['pre1920'] + row['G_all']
        elif year >= 1920 and year <= 1941:
            pos_dict[ID]['1920-41'] = pos_dict[ID]['1920-41'] + row['G_all']
        elif year >= 1942 and year <= 1945:
            pos_dict[ID]['1942-45'] = pos_dict[ID]['1942-45'] + row['G_all']
        elif year >= 1946 and year <= 1962:
            pos_dict[ID]['1946-62'] = pos_dict[ID]['1946-62'] + row['G_all']
        elif year >= 1963 and year <= 1976:
            pos_dict[ID]['1963-76'] = pos_dict[ID]['1963-76'] + row['G_all']
        elif year >= 1977 and year <= 1992:
            pos_dict[ID]['1977-92'] = pos_dict[ID]['1977-92'] + row['G_all']
        elif year >= 1993 and year <= 2009:
            pos_dict[ID]['1993-2009'] = pos_dict[ID]['1993-2009'] + row['G_all']
        elif year > 2009:
            pos_dict[ID]['post2009'] = pos_dict[ID]['post2009'] + row['G_all']
    else:
        pos_dict[ID] = {}
        pos_dict[ID]['G_all'] = row['G_all']
        pos_dict[ID]['G_p'] = row['G_p']
        pos_dict[ID]['G_c'] = row['G_c']
        pos_dict[ID]['G_1b'] = row['G_1b']
        pos_dict[ID]['G_2b'] = row['G_2b']
        pos_dict[ID]['G_3b'] = row['G_3b']
        pos_dict[ID]['G_ss'] = row['G_ss']
        pos_dict[ID]['G_lf'] = row['G_lf']
        pos_dict[ID]['G_cf'] = row['G_cf']
        pos_dict[ID]['G_rf'] = row['G_rf']
        pos_dict[ID]['G_of'] = row['G_of']
        pos_dict[ID]['G_dh'] = row['G_dh']
        pos_dict[ID]['pre1920'] = 0
        pos_dict[ID]['1920-41'] = 0
        pos_dict[ID]['1942-45'] = 0
        pos_dict[ID]['1946-62'] = 0
        pos_dict[ID]['1963-76'] = 0
        pos_dict[ID]['1977-92'] = 0
        pos_dict[ID]['1993-2009'] = 0
        pos_dict[ID]['post2009'] = 0
        
        if year < 1920:
            pos_dict[ID]['pre1920'] = row['G_all']
        elif year >= 1920 and year <= 1941:
            pos_dict[ID]['1920-41'] = row['G_all']
        elif year >= 1942 and year <= 1945:
            pos_dict[ID]['1942-45'] = row['G_all']
        elif year >= 1946 and year <= 1962:
            pos_dict[ID]['1946-62'] = row['G_all']
        elif year >= 1963 and year <= 1976:
            pos_dict[ID]['1963-76'] = row['G_all']
        elif year >= 1977 and year <= 1992:
            pos_dict[ID]['1977-92'] = row['G_all']
        elif year >= 1993 and year <= 2009:
            pos_dict[ID]['1993-2009'] = row['G_all']
        elif year > 2009:
            pos_dict[ID]['post2009'] = row['G_all']

In [37]:
pos_df = pd.DataFrame.from_dict(pos_dict,orient='index')

In [38]:
pos_df.columns

Index(['G_all', 'G_p', 'G_c', 'G_1b', 'G_2b', 'G_3b', 'G_ss', 'G_lf', 'G_cf',
       'G_rf', 'G_of', 'G_dh', 'pre1920', '1920-41', '1942-45', '1946-62',
       '1963-76', '1977-92', '1993-2009', 'post2009'],
      dtype='object')

In [39]:
pos_df.head()

,G_all,G_p,G_c,G_1b,G_2b,G_3b,G_ss,G_lf,G_cf,G_rf,G_of,G_dh,pre1920,1920-41,1942-45,1946-62,1963-76,1977-92,1993-2009,post2009
aardsda01,331,331,0,0,0,0,0,0,0,0,0,0.0,0,0,0,0,0,0,201,130
aaronha01,3298,0,0,210,43,7,0,315,308,2174,2760,201.0,0,0,0,1350,1948,0,0,0
aaronto01,437,0,0,232,7,10,0,135,1,2,137,0.0,0,0,0,141,296,0,0,0
aasedo01,448,448,0,0,0,0,0,0,0,0,0,0.0,0,0,0,0,0,448,0,0
abadan01,15,0,0,8,0,0,0,0,0,1,1,0.0,0,0,0,0,0,0,15,0


join pos_df and master_df

In [40]:
master_df = master_df.join(pos_df,on='playerID',how='right')
master_df.head()

,playerID,nameFirst,nameLast,bats,throws,debut,finalGame,G,AB,R,...,G_of,G_dh,pre1920,1920-41,1942-45,1946-62,1963-76,1977-92,1993-2009,post2009
0,aardsda01,David,Aardsma,R,R,2004-04-06,2015-08-23,331,4,0,...,0,0.0,0,0,0,0,0,0,201,130
1,aaronha01,Hank,Aaron,R,R,1954-04-13,1976-10-03,3298,12364,2174,...,2760,201.0,0,0,0,1350,1948,0,0,0
2,aaronto01,Tommie,Aaron,R,R,1962-04-10,1971-09-26,437,944,102,...,137,0.0,0,0,0,141,296,0,0,0
3,aasedo01,Don,Aase,R,R,1977-07-26,1990-10-03,448,5,0,...,0,0.0,0,0,0,0,0,448,0,0
4,abadan01,Andy,Abad,L,L,2001-09-10,2006-04-13,15,21,1,...,1,0.0,0,0,0,0,0,0,15,0


In [48]:
master_df.columns

Index(['playerID', 'nameFirst', 'nameLast', 'bats', 'throws', 'debut',
       'finalGame', 'G', 'AB', 'R', 'H', '2B', '3B', 'HR', 'RBI', 'SB', 'BB',
       'SO', 'IBB', 'HBP', 'SH', 'SF', 'Years_Played', 'Gf', 'GSf', 'POf',
       'Af', 'Ef', 'DPf', 'HoF', 'votedBy', 'Most Valuable Player', 'AS_games',
       'Gold Glove', 'Rookie of the Year', 'World Series MVP',
       'Silver Slugger', 'playerIDmstr', 'G_all', 'G_p', 'G_c', 'G_1b', 'G_2b',
       'G_3b', 'G_ss', 'G_lf', 'G_cf', 'G_rf', 'G_of', 'G_dh', 'pre1920',
       '1920-41', '1942-45', '1946-62', '1963-76', '1977-92', '1993-2009',
       'post2009'],
      dtype='object')

**feature and target columns**

In [51]:
num_cols = ['playerID', 'nameFirst', 'nameLast', 'HoF', 
            'H', 'BB', 'HR',  'RBI',
            'R', 'SB', '2B', '3B', 'AB', 'SO', 'Most Valuable Player', 
            'World Series MVP', 'AS_games','Gold Glove', 
            'Rookie of the Year', 'Silver Slugger', 
            'DPf', 'Af', 'Ef', 'G_all']

In [58]:
data = master_df[num_cols]
data = data.fillna(0)
target = data['HoF']
features = data.drop(['playerID','nameFirst', 'nameLast', 'HoF'],
                      axis=1)

**logistic regression**

In [44]:
from sklearn.cross_validation import cross_val_predict,KFold
from sklearn.linear_model import LogisticRegression

/Users/ruihu/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [45]:
lr = LogisticRegression(class_weight='balanced')

In [59]:
kf = KFold(features.shape[0],random_state=1)

In [60]:
predictions_lr = cross_val_predict(lr,features,target,cv=kf)

**error metric**

In [66]:
import numpy as np

In [67]:
np_predictions_lr = np.asarray(predictions_lr)
np_target = target.as_matrix()

/Users/ruihu/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  


determine true positive count

In [68]:
tp_filter_lr = (np_predictions_lr==1)&(np_target==1)
tp_lr = len(np_predictions_lr[tp_filter_lr])

determine false negative count

In [69]:
fn_filter_lr = (np_predictions_lr==0)&(np_target==1)
fn_lr = len(np_predictions_lr[fn_filter_lr])

determine true negative count

In [70]:
tn_filter_lr = (np_predictions_lr==0)&(np_target==0)
tn_lr = len(np_predictions_lr[tn_filter_lr])

determine false positive count

In [72]:
fp_filter_lr = (np_predictions_lr==1)&(np_target==0)
fp_lr = len(np_predictions_lr[fp_filter_lr])

determine tp, fn, fp rate

In [73]:
tpr = tp_lr/(tp_lr + fn_lr)
fnr = fn_lr/(fn_lr + tp_lr)
fpr = fp_lr/(fp_lr + tn_lr)

In [74]:
print(tpr,fnr,fpr)

0.875 0.125 0.058272688356164386


This model is fair accurate with relatively high false negative rate.

**Conclusion**

In this second part of the project, I imported data from several csv files, cleaned and aggregated players statistics, applied logistic regression model to predict which players will be voted into the Hall of Fame. 